## How to Use this File?

make sure added your personal bearer_token, define the path of the data and save path of the tweets.

DATA_PATH is the destination of a txt file containing comma-separated twitter IDs.
JSON_PATH is the save location of the crawlling tweets, it is a json file containing a list of tweets

def Extend_Json: the function takes a list of dictionary as input, to extend the destination json file without reading it into memory

def get_tweet: use API to get twitter from the web, it separates a list of IDs into 100-sized list chunks, so the number of requests sent should be 

#Requests = #ofIDs/100


In [44]:
import tweepy
import numpy as np
import time
import json
import os

In [45]:
DATA_PATH = "./id_data/train.data.txt"   # train
#DATA_PATH = "./id_data/dev.data.txt"  # dev

AUTHOR_JSON_PATH = "./full_data/source_author_ids.json"

SOURCE_STORY_ONLY = True
if SOURCE_STORY_ONLY:
    JSON_PATH = "./full_data/full_train_source_only.json"   # SOURCE_STORY_ONLY=True
else:
    JSON_PATH = "./full_data/full_train.json"


## Twitter API V2.0

In [46]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJzFbwEAAAAAiw1m4S%2FgkghMs%2BWfiVo6OswWgYE%3DGRQPPybZpQuJ1g2lFvxyH9yogZRQatwb6gT8X8ShhyYdjmqXGu"

client = tweepy.Client(bearer_token)

## Twitter API V1.1

In [47]:
## Twitter API V1.1
consumer_key = "PCyRF0jWO12cgQ88uQDkDoBes"  # API_Key and secret
consumer_secret = "whVymp4fEIeKQEy1hrDghwRxKyW515QeUzFKq5fMvryoe1mOxT"
access_token = "1357932194019217410-RmO2xXuajMNs98s6qw5DYHLAFzMcLZ"
access_token_secret = "ptzYrYJF10qnGf8vU9wPdiuRNOAM3BXFxjMhOcxBZZcKz"

auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret
)
api = tweepy.API(auth)
# If the authentication was successful, this should print the
# screen name / username of the account
print(api.verify_credentials().screen_name)

FAAT_JOEY


In [48]:
def Extend_Json(new_list_dict, File_PATH):
    """Extend a huge json file without loading it to the memory, input: new_dict is a list of json dictionary objects"""
    # if file exists:
    if os.path.exists(File_PATH):
        if os.stat(File_PATH).st_size == 0:
            print("asasdasdasd")
            # if file contains nothing, delete the file
            os.remove(File_PATH)
            # create another new json file
            with open (File_PATH, mode="w") as file:
                json.dump(new_list_dict, file, indent=2)
        else:
            if new_list_dict: # if dict has value
                with open (File_PATH, mode="r") as file:
                    data = json.load(file)
                # Source: https://stackoverflow.com/questions/18087397/append-list-of-python-dictionaries-to-a-file-without-loading-it/31224105#31224105?newreg=6b7713ed96df42959bb9443daf7bb8ec
                # two methods
                # with open (File_PATH, mode="r+") as file:
                #     file.seek(0,2)  # 0 byte (0) from the end of file (2)
                #     position = file.tell() -1
                #     file.seek(position)
                #     file.write( ",{}]".format(json.dumps(new_dict)[1:-1]) )
                with open (File_PATH, mode="r+") as file:
                    file.seek(os.stat(File_PATH).st_size -1)
                    file.write( ",{}]".format(json.dumps(new_list_dict, indent=2)[1:-1]) )
    else:
        # else, create a new json file
        with open (File_PATH, mode="w") as file:
            json.dump(new_list_dict, file, indent=2)

## Twitter Part

In [49]:
total_id_list = []
with open(DATA_PATH, "r") as f:
    for line in f:
        line = line.strip() # remove next line
        if SOURCE_STORY_ONLY:
            line = line.split(',')[0] # split into list\
        else:
            line = line.split(',')
        total_id_list.append(line)
if not SOURCE_STORY_ONLY:
    total_id_list = [item for sublist in total_id_list for item in sublist] # Flat into a single list
total_id_list = list(set(total_id_list))
len(total_id_list)

1887

In [50]:
def get_twitter(client, list_ids, JSON_PATH, tweet_fields=["created_at", "author_id", "entities", "withheld", "public_metrics", "lang", "reply_settings", "source"]):
    """Automatically get a list of tweeters, the function auto divide ids into chunks of size 800 to avoid excessive use of API,
        auto create a Json file in the designated path."""
    CHUNK_SIZE = 99
    ## Auto divide into sub-tasks
    list_ids = list(set(list_ids))
    if len(list_ids) > CHUNK_SIZE:
        chunks_of_list = np.array_split(list_ids, int(len(list_ids)/CHUNK_SIZE)+1)
    else:
        chunks_of_list = list_ids
    count = 0
    
    for sub_list in chunks_of_list:
        ## Main loop to get twitter by chunks
        output_list = []   # output list of dictionary of tweets
        author_id_list = []  # store author ids to produce author database

        tweet_ids = list(sub_list)
        
        # This endpoint/method returns a variety of information about the Tweet(s)
        # specified by the requested ID or list of IDs
        # By default, only the ID and text fields of each Tweet will be returned
        # Additional fields are retrieved using the tweet_fields parameter, selected by hands to make sure we capture useful informations
        response = client.get_tweets(tweet_ids, tweet_fields=tweet_fields)

        for tweet in response.data:
            # set withheld to true or false based on return
            if tweet.withheld:
                withheld = True
            else:
                withheld = False
            tweet_dict = {
                "id":              tweet.id,
                "created_at":      str(tweet.created_at),
                "author_id":       tweet.author_id, 
                "lang":            tweet.lang,
                "reply_settings":  tweet.reply_settings,
                "source":          tweet.source,
                "withheld":        withheld,
                "public_metrics":  tweet.public_metrics,
                "entities":        tweet.entities,
                "text":            tweet.text,
            }
            output_list.append(tweet_dict)
            author_id_list.append(tweet.author_id) # record author informations as well
            count += 1
        
        # Save to json file
        Extend_Json(output_list, JSON_PATH)
        Extend_Json(author_id_list, AUTHOR_JSON_PATH)
        # Sleep 15 minutes to avoid API restrictions after processing 900 tweets
        #if loop_count == 8: 
            #time.sleep(15*60)
            #loop_count = 0
    print("Total Tweeter :", count)
        

In [51]:
# Run getter.
get_twitter(client, total_id_list, JSON_PATH)

Total Tweeter : 1554


In [52]:
with open(JSON_PATH, "r") as f:
    data = json.load(f)
    print("Number of twitter stored: ", len(data))

Number of twitter stored:  1554


In [53]:
print("Number of Source Story crawled: ",len(data)/28633)
#print("Number of Total Training Twitter crawled: ", 366/28633)

Number of Source Story crawled:  0.054273041595362


## Author Data Part

Extract Author data for the source twitter and output into AUTHOR_DATA_PATH

In [58]:
## Only get source author's data for now
SOURCE_AUTHOR_PATH = "./full_data/data_storage/source_author_ids.json"
AUTHOR_DATA_PATH = "./full_data/source_author_data.json"

In [59]:
with open(SOURCE_AUTHOR_PATH) as f:
    author_data = json.load(f)
    author_list = list(set(author_data))
len(author_list)

1388

In [60]:
def get_author_data(api, id_list, OUTPUT_PATH):
    """Extract Author data for a list of authors and output them into AUTHOR_DATA_PATH as a json"""
    list_of_authors = []
    count = 0
    for author_id in author_data:
        response = api.get_user(user_id=author_id)
        list_of_authors.append(response._json)
        count += 0
        if (count>=800) and (count % 800 == 0):
            # Sleep 15 minutes to avoid limits
            time.sleep(1000)
    ## Convert to json        
    Extend_Json(list_of_authors, AUTHOR_DATA_PATH)
    print("Total Author Got: ", count)

In [62]:
get_author_data(api, author_list, AUTHOR_DATA_PATH)

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded